# 草稿

In [1]:
from utils.tvm import set_env
from vta.top import graph_pack
from tvm import autotvm, relay

Python: 3.9.12
TVM 根目录： /media/pc/data/4tb/lxw/study/tvm


## 用 Relay 编译模型

首先使用导入器将模型导入到 relay。然后，将模型与标准优化一起构建成 TVM 库。最后，从该库中创建 TVM 图运行时模块。

In [ ]:
import onnx
from tvm import relay
from tvm.contrib import graph_executor

# 加载模型
model_path = '../../models/resnet50-v2-7.onnx'
onnx_model = onnx.load(model_path)

target = "llvm"

# 不同的模型类型，输入的名称可能不同。
# 你可以使用 Netron 这样的工具来检查输入名称
input_name = "data"
shape_dict = {input_name: img_data.shape}

# 前端导入模型
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 编译模型
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target=target, params=params)

dev = tvm.device(target, 0)
module = graph_executor.GraphModule(lib["default"](dev))

## 在 TVM 运行时上执行

要使用 TVM 来运行模型并进行预测，我们需要两样东西：

- 编译后的模型 `module`
- 对模型的有效输入，以便进行预测

In [ ]:
dtype = "float32"
module.set_input(input_name, img_data)
module.run()
output_shape = (1, 1000)
tvm_output = module.get_output(0, tvm.nd.empty(output_shape)).numpy()

## 收集基本性能数据

想收集一些与这个未优化的模型相关的基本性能数据，并在以后与调优后的模型进行比较。为了帮助说明 CPU 的噪音，在多个批次的重复中运行计算，然后收集一些关于平均值、中位数和标准差的基础统计数据。

In [ ]:
import timeit
import numpy as np

timing_number = 10
timing_repeat = 10
unoptimized = (
    np.array(timeit.Timer(lambda: module.run()).repeat(repeat=timing_repeat, number=timing_number))
    * 1000
    / timing_number
)
unoptimized = {
    "mean": np.mean(unoptimized),
    "median": np.median(unoptimized),
    "std": np.std(unoptimized),
}

print(unoptimized)

## 对输出进行后处理

将 ResNet-50 v2 的输出渲染成更适合人类阅读的形式。

In [ ]:
from scipy.special import softmax
from tvm.contrib.download import download_testdata

# Download a list of labels
labels_url = "https://s3.amazonaws.com/onnx-model-zoo/synset.txt"
labels_path = download_testdata(labels_url, "synset.txt", module="data")

with open(labels_path, "r") as f:
    labels = [l.rstrip() for l in f]

# Open the output and read the output tensor
scores = softmax(tvm_output)
scores = np.squeeze(scores)
ranks = np.argsort(scores)[::-1]
for rank in ranks[0:5]:
    print("class='%s' with probability=%f" % (labels[rank], scores[rank]))

In [ ]:
import tvm
import tvm.testing
from tvm import te
import numpy as np

In [ ]:
import tvm

In [ ]:
conda install tvm